In [2]:
import torch
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import AutoTokenizer, DataCollatorWithPadding
from datasets import Dataset
from datasets import load_metric

from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

from scipy.stats import mode
import os
os.environ['WANDB_DISABLED'] = 'True'

In [6]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train, valid = train_test_split(train, test_size=0.2, random_state=2)
train.head(10)

,id,keyword,location,text,target
4549,6466,injured,USA,Offers : http://t.co/Gl3C1vc88P #8392 Deluxe T...,1
4512,6413,hurricane,NaN,The hurricane mixxtail kinda tastes like the w...,0
4368,6203,hijacker,NaN,Complete Solution to Get Rid of http://t.co/9C...,0
4297,6103,hellfire,NaN,@HellFire_eV @JackPERU1 then I do this to one ...,0
13,19,NaN,NaN,#Flood in Bago Myanmar #We arrived Bago,1
6235,8903,snowstorm,Manchester,@Groupon_UK it won't let me as you don't follo...,0
3160,4537,emergency,Southern Maine,Former heroin addict shares story as city lead...,1
2917,4191,drown,somewhere in Indiana,Going to go drown my sorrows with sad music brb,0
2318,3334,demolished,Chicago,ÛÏ@SplottDave: @TeamPalestina That's about 28...,1
3392,4856,evacuation,NaN,This is an evil generation\nRock and roll evac...,0


In [7]:
test.head(10)

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
5,12,NaN,NaN,We're shaking...It's an earthquake
6,21,NaN,NaN,They'd probably still show more life than Arse...
7,22,NaN,NaN,Hey! How are you?
8,27,NaN,NaN,What a nice hat?
9,29,NaN,NaN,Fuck off!


In [8]:
def tokenization(model_path, train_df, valid_df, test_df):
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    train_ds = Dataset.from_pandas(train_df)
    valid_ds = Dataset.from_pandas(valid_df)
    test_ds = Dataset.from_pandas(test_df)

    def process_token(example, tokenizer=tokenizer):
        return tokenizer(example['text'])
    
    tokenized_train = train_ds.map(process_token)
    tokenized_valid = valid_ds.map(process_token)
    tokenized_test = test_ds.map(process_token)

    columns_to_remove = ['id', 'keyword', 'location', '__index_level_0__']
    train_dataset = tokenized_train.remove_columns(columns_to_remove)
    valid_dataset = tokenized_valid.remove_columns(columns_to_remove)
    columns_to_remove_test = ['id', 'keyword', 'location']
    test_dataset = tokenized_test.remove_columns(columns_to_remove_test)

    train_dataset = train_dataset.rename_column('target', 'label')
    valid_dataset = valid_dataset.rename_column('target', 'label')
    return train_dataset, valid_dataset, test_dataset, tokenizer

In [10]:
def compute_metrics(eval_pred):
    load_acc = load_metric('accuracy')
    load_f1 = load_metric('f1')
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    acc = load_acc.compute(predictions=predictions, refereences = labels)['accuracy']
    f1 = load_f1.compute(predictions=predictions, references = labels)['f1']
    return {'acc':acc, 'f1':f1}

In [19]:
def init_trainer(model_path, tokenizer, lr, ep, train_dataset, valid_dataset):
    model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2)
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    train_args = TrainingArguments(
        learning_rate=lr,
        num_train_epochs=ep,
        per_device_train_batch_size=16,
        weight_decay=0.01,
        output_dir=model_path
    )

    trainer = Trainer(
        model=model,
        args=train_args,
        train_dataset=train_dataset,
        eval_dataset=valid_dataset,
        data_collator=data_collator,
        compute_metrics=compute_metrics
    )

    return trainer

In [16]:
class model_fusion:
    def __init__(self, model_paths, lr, ep) -> None:
        self.model_paths= model_paths
        self.learning_rate = lr
        self.epoch_num = ep
        self.preds =[]

    def train_pred_multiple_models(self):
        for model_path in self.model_paths:
            print(f'training : {model_path}')
            print('total:', torch.cuda.get_device_properties(0).total_memory/1e9)
            print('allocated:', torch.cuda.memory_allocated(0)/1e9)
            print('cached', torch.cuda.memory_reserved(0)/1e9)

            train_dataset, valid_dataset, test_dataset, tokenizer = tokenization(model_path, train, valid, test)
            trainer = init_trainer(model_path, tokenizer, self.learning_rate, self.epoch_num, train_dataset, valid_dataset)
            trainer.train()

            prediction = trainer.predict(test_dataset=test_dataset)
            predictions = np.argmax(a=prediction.predictions, axis=-1)
            self.preds.append(model_path, predictions)

    def fusion_pred(self):
        all_preds = [pred[1] for pred in self.preds]
        final_preds = mode(all_preds, axis=0)[0]

        return final_preds.ravel()

In [24]:
roberta_path = 'roberta-base'
deberta_path = 'microsoft/deberta-v3-base'
distilbert_path = 'distilbert-base-uncased'
model_paths = [distilbert_path,roberta_path,deberta_path]
lr = 2e-5
ep = 3
models = model_fusion(model_paths,lr,ep)

In [25]:
models.train_pred_multiple_models()
pred = models.fusion_pred()
models.preds

training : distilbert-base-uncased
total: 25.756696576
allocated: 1.3396864
cached 2.764046336


d:\Anaconda\envs\pytorch\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\paulw\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Map: 100%|██████████| 3263/3263 [00:00<00:00, 9952.91 examples/s] 
Some weights of DistilBertForSequenceClassificatio

{'loss': 0.4222, 'grad_norm': 5.252051830291748, 'learning_rate': 1.1251093613298338e-05, 'epoch': 1.31}


 87%|████████▋ | 1000/1143 [00:21<00:02, 51.74it/s]

{'loss': 0.3019, 'grad_norm': 1.9004838466644287, 'learning_rate': 2.502187226596676e-06, 'epoch': 2.62}


100%|██████████| 1143/1143 [00:27<00:00, 41.54it/s]

{'train_runtime': 27.5137, 'train_samples_per_second': 664.032, 'train_steps_per_second': 41.543, 'train_loss': 0.3481059704344208, 'epoch': 3.0}


KeyError: 0

In [26]:
def tokenization(model_path,train_df,valid_df,test_df):
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    train_ds = Dataset.from_pandas(train_df)
    valid_ds = Dataset.from_pandas(valid_df)
    test_ds = Dataset.from_pandas(test_df)
    
    def process_token(example,tokenizer=tokenizer):
        return tokenizer(example['text'])
    tokenized_train = train_ds.map(process_token)
    tokenized_valid = valid_ds.map(process_token)
    tokenized_test = test_ds.map(process_token)
    
    columns_to_remove = ['id', 'keyword', 'location','__index_level_0__']
    train_dataset = tokenized_train.remove_columns(columns_to_remove)
    valid_dataset = tokenized_valid.remove_columns(columns_to_remove)
    columns_to_remove = ['id', 'keyword', 'location']
    test_dataset = tokenized_test.remove_columns(columns_to_remove)
    
    train_dataset = train_dataset.rename_column("target", "label")
    valid_dataset = valid_dataset.rename_column("target", "label")
    return train_dataset,valid_dataset,test_dataset,tokenizer

In [27]:
def compute_metrics(eval_pred):
    load_acc = load_metric('accuracy')
    load_f1 = load_metric('f1')
    logits,labels = eval_pred
    predictions = np.argmax(logits,axis = -1)
    acc = load_acc.compute(predictions = predictions,references = labels)['accuracy']
    f1 = load_f1.compute(predictions = predictions, references = labels)['f1']
    return {'acc':acc,'f1':f1}

In [28]:
def init_trainer(model_path,tokenizer,lr,ep,train_dataset,valid_dataset):
    model = AutoModelForSequenceClassification.from_pretrained(model_path,num_labels=2)
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    training_args = TrainingArguments(
        learning_rate=lr,
        num_train_epochs=ep,
        per_device_train_batch_size=16,
        weight_decay=0.01,
        output_dir=model_path,

    )
    trainer = Trainer(
        model = model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=valid_dataset,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )
    return trainer

In [29]:
class model_fusion:
    def __init__(self,model_paths,lr,ep):
        self.model_paths = model_paths
        self.learning_rate = lr
        self.epoch_num = ep
        self.preds = []
    
    def train_pred_multiple_models(self):
        for model_path in self.model_paths:
            print(f'Training {model_path}')
            print('total:',torch.cuda.get_device_properties(0).total_memory/1e9)
            print('allocated:',torch.cuda.memory_allocated(0)/1e9)
            print('cached:',torch.cuda.memory_reserved(0)/1e9)
            train_dataset,valid_dataset,test_dataset,tokenizer = tokenization(model_path,train,valid,test)
            trainer = init_trainer(model_path,tokenizer,self.learning_rate,self.epoch_num,train_dataset,valid_dataset)
            trainer.train()
            prediction = trainer.predict(test_dataset=test_dataset)
            predictions = np.argmax(a=prediction.predictions,axis = -1)
            self.preds.append((model_path,predictions))
            
            
    def fusion_pred(self):
        all_preds = [pred[1] for pred in self.preds]
        final_preds = mode(all_preds, axis=0)[0]

        return final_preds.ravel()

In [34]:
roberta_path = 'roberta-base'
deberta_path = 'microsoft/deberta-v3-base'
model_paths = [roberta_path,deberta_path]
lr = 2e-5
ep = 3
models = model_fusion(model_paths,lr,ep)

In [35]:
models.train_pred_multiple_models()
pred = models.fusion_pred()
models.preds

Training roberta-base
total: 25.756696576
allocated: 0.836468224
cached: 1.696595968


d:\Anaconda\envs\pytorch\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\paulw\.cache\huggingface\hub\models--roberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Map: 100%|██████████| 3263/3263 [00:00<00:00, 9845.93 examples/s]
Some weights of RobertaForSequenceClassification were not init

{'loss': 0.425, 'grad_norm': 3.327252149581909, 'learning_rate': 1.1251093613298338e-05, 'epoch': 1.31}


 87%|████████▋ | 1000/1143 [00:40<00:05, 27.90it/s]

{'loss': 0.3069, 'grad_norm': 9.085705757141113, 'learning_rate': 2.502187226596676e-06, 'epoch': 2.62}


...

d:\Anaconda\envs\pytorch\Lib\site-packages\transformers\convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Map: 100%|██████████| 3263/3263 [00:00<00:00, 8045.56 examples/s]
d:\Anaconda\envs\pytorch\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of DebertaV2ForSequenceCl

{'loss': 0.433, 'grad_norm': 2.5981009006500244, 'learning_rate': 1.1251093613298338e-05, 'epoch': 1.31}


 87%|████████▋ | 1000/1143 [1:21:31<00:08, 16.21it/s]   

{'loss': 0.3106, 'grad_norm': 3.2711493968963623, 'learning_rate': 2.502187226596676e-06, 'epoch': 2.62}


KeyboardInterrupt: 